In [2]:
] activate .

In [2]:
using Images
using Metalhead
using Metalhead: classify
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Images
using Random

In [3]:
function get_images(dir,num)
    images= []
    for (root, dirs, files) in walkdir(dir)    
        for file in files[1:num]
            f_name=joinpath(root,file)
            pic=load(f_name)
            append!(images,[pic])
        end
    end
    return images
end


get_images (generic function with 1 method)

In [4]:
function run_vgg()
    vgg=VGG19()
    v=[]
    c=0
    images=get_images("flowers-recognition/flowers/daisy/",100)
    for pic in images
        pred=classify(vgg,pic)
        if pred!="daisy"
            c+=1
        end
    end
    return c
end
    
    

run_vgg (generic function with 1 method)

In [15]:
mistakes=run_vgg()

UndefVarError: UndefVarError: VGG19 not defined

In [16]:
vgg_26 = Chain(
    VGG19().layers[1:26],
    Dense(4096,2),
    softmax
)

Chain(Chain(Conv((3, 3), 3=>64, NNlib.relu), Conv((3, 3), 64=>64, NNlib.relu), getfield(Metalhead, Symbol("##42#48"))(), Conv((3, 3), 64=>128, NNlib.relu), Conv((3, 3), 128=>128, NNlib.relu), getfield(Metalhead, Symbol("##43#49"))(), Conv((3, 3), 128=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), getfield(Metalhead, Symbol("##44#50"))(), Conv((3, 3), 256=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), getfield(Metalhead, Symbol("##45#51"))(), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), getfield(Metalhead, Symbol("##46#52"))(), getfield(Metalhead, Symbol("##47#53"))(), Dense(25088, 4096, NNlib.relu), Dropout{Float32}(0.5f0, false), Dense(4096, 4096, NNlib.relu), Dropout{Float32}(0.5f0, false)), Dense(4096, 2), NNlib.softmax)

In [5]:
function get_rose_sunflower()
    rose_100_img= get_images("flowers-recognition/flowers/rose/",100)
    sunflower_100_img= get_images("flowers-recognition/flowers/sunflower/",100)
    rose_train, rose_test= rose_100_img[1:80],rose_100_img[81:100]
    sunflower_train, sunflower_test= sunflower_100_img[1:80],sunflower_100_img[81:100]
    
    train_x,train_y = hcat(rose_train,sunflower_train),hcat(repeat(["rose"],80) , repeat(["sunflower"],80) )
    test_x,test_y = vcat(rose_test,sunflower_test),vcat(repeat(["rose"],20) , repeat(["sunflower"],20) )
    
    index_shuff= shuffle(Vector(1:160))
    train_shuff_x,train_shuff_y= train_x[index_shuff],train_y[index_shuff]
    
    #Preprocessing the dataset using metalhead preprocessing
    train_shuff_x=[(Metalhead.preprocess(train_shuff_x[i])) for i in range(1,length=size(train_shuff_x,1))] 
    test_x=[(Metalhead.preprocess(test_x[i])) for i in range(1,length=size(test_x,1))] 
    return train_shuff_x,train_shuff_y,test_x,test_y
end

get_rose_sunflower (generic function with 1 method)

In [17]:
loss(x, y) = crossentropy(vgg_26(x), y)
accuracy(x, y) = mean(onecold(vgg_26(x)) .== onecold(y))

function train_VGG26(X, Y, dataset,n)
    evalcb = throttle(() -> @show(accuracy(X, Y)), 10)
    opt = ADAM(params(vgg_26))
    for i=1:n
        Flux.train!(loss, dataset, opt, cb = evalcb)
    end
end

function test_VGG26(X, Y)
    accuracy(X, Y)
end

test_VGG26 (generic function with 1 method)

In [6]:
train_x,train_y,test_x,test_y=get_rose_sunflower()
train_y = onehotbatch(train_y, ["rose" , "sunflower"]);
test_y = onehotbatch(test_y, ["rose" , "sunflower"]);

train_x_all=train_x[1]
for i=2:size(train_x,1)
    train_x_all= cat(train_x_all,train_x[i]; dims=4)
end


test_x_all= test_x[1]
for i=2:size(test_x,1)
    test_x_all= cat(test_x_all,test_x[i]; dims=4)
end


In [15]:
#train_x_all = convert(Array{Float64,4}, train_x_all);
batches=[(train_x_all[:,:,:,i],train_y[:,i]) for i in partition(1:size(train_x_all,4),10)] ;

In [18]:
train_VGG26(train_x_all,train_y,batches,1)

accuracy(X, Y) = 0.5875
accuracy(X, Y) = 0.8125
accuracy(X, Y) = 0.91875
accuracy(X, Y) = 0.90625
accuracy(X, Y) = 0.9
accuracy(X, Y) = 0.9375


In [19]:
test_VGG26(test_x_all,test_y)

0.75

1. Make a new chain called "vgg_24" that takes the first 24 layers from the VGG as they are. and add to them
3 layers: Dense layer, and Another Dense layer to classify the two classes and a softmax layer.
2. In this chain, Initlize the w and b of the first Dense that youy just created by the optimal weights that VGG
reached in it before.

Note
This Dense layer is called in VGG "fc7" and you can get the optimal weights from :
Metalhead.weights()..Search for this on the github of Metahead and you can find an example to
run over VGG.
Then get the w and b of the layer called fc7 and initlize your first dense with it to start our
learning from these values

3.Train and Test vgg_24 in same way as in question 2 and record the testing accuracy

In [7]:
Pkg.add("BSON") 
using BSON

  Updating registry at `/home/jrun/.julia/registries/JuliaPro`
  Updating git-repo `https://pkg.juliacomputing.com/registry/JuliaPro`
Fetching: [========================================>]  100.0 %.0 %13.7 %>                             ]  27.4 %                  ]  54.7 % [============================>            ]  68.4 % ]  95.3 % Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]


In [8]:
ws=BSON.load("vgg19.bson")

Dict{Symbol,Any} with 38 entries:
  :conv4_2_w_0 => Float32[-0.00605492 -0.00096373 0.000626743; 0.00679103 0.009…
  :conv3_3_w_0 => Float32[-0.000198241 -0.000861549 -0.00059339; 0.00153241 -0.…
  :fc6_w_0     => Float32[0.000740605 -9.02351e-5 … 0.000489794 -0.000968067; -…
  :fc8_w_0     => Float32[0.0117621 -0.00695795 … -0.00535216 0.0103952; -0.011…
  :conv4_3_b_0 => Float32[0.00169088, 0.043703, 0.0566955, 0.0205713, 0.0076408…
  :conv1_2_w_0 => Float32[0.148688 0.170782 -0.0197718; 0.181252 -0.0287904 -0.…
  :conv3_4_w_0 => Float32[-0.0145519 -0.0260602 -0.0111239; -0.00908116 -0.0275…
  :conv2_2_w_0 => Float32[-0.000591473 -0.00272857 -0.00527541; 0.00139092 0.00…
  :conv2_2_b_0 => Float32[-0.0764114, -0.110066, -0.144156, 0.0385552, 0.258943…
  :conv3_3_b_0 => Float32[0.0205515, -0.0192079, 0.0514344, 0.0547953, 0.094897…
  :conv2_1_b_0 => Float32[-0.0426487, 0.0790341, -0.0259495, 0.099568, 0.113474…
  :conv4_1_w_0 => Float32[0.0258524 0.0609968 0.0256101; -0.00310215 -0.001

In [9]:
dense_f7_w=ws[:fc7_w_0]'
dense_f7_b=ws[:fc7_b_0]

4096-element Array{Float32,1}:
 0.63382286
 0.4577255 
 0.57202035
 0.37137407
 0.5638934 
 0.42521483
 0.24163638
 0.18872565
 0.6788014 
 0.3629831 
 0.21210502
 0.6111959 
 0.41938317
 ⋮         
 0.51462954
 0.5900871 
 0.47841918
 0.29396743
 0.4824054 
 0.3367087 
 0.46477816
 0.5200556 
 0.09609667
 0.5036738 
 0.41717583
 0.6895467 

In [10]:
# dense_layer_w= params(m)[3]
# dense_layer_b= params(m)[4]

# function my_init_w(out,in_n)
#     Tracker.data(dense_layer_w)
# end

function my_init_w(out,in_n)
    return dense_f7_w
end

function my_init_b(b)
    return dense_f7_b
end


my_init_b (generic function with 1 method)

In [43]:
vgg_24 = Chain(
    VGG19().layers[1:24],
    Dense(4096, 4096, relu, initW= my_init_w, initb= my_init_b),
    Dense(4096, 2),
    softmax
)

Chain(Chain(Conv((3, 3), 3=>64, NNlib.relu), Conv((3, 3), 64=>64, NNlib.relu), getfield(Metalhead, Symbol("##42#48"))(), Conv((3, 3), 64=>128, NNlib.relu), Conv((3, 3), 128=>128, NNlib.relu), getfield(Metalhead, Symbol("##43#49"))(), Conv((3, 3), 128=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), getfield(Metalhead, Symbol("##44#50"))(), Conv((3, 3), 256=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), getfield(Metalhead, Symbol("##45#51"))(), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), getfield(Metalhead, Symbol("##46#52"))(), getfield(Metalhead, Symbol("##47#53"))(), Dense(25088, 4096, NNlib.relu), Dropout{Float32}(0.5f0, false)), Dense(4096, 4096, NNlib.relu), Dense(4096, 2), NNlib.softmax)

In [44]:
loss(x, y) = crossentropy(vgg_24(x), y)
accuracy(x, y) = mean(onecold(vgg_24(x)) .== onecold(y))

function train_VGG24(X, Y, dataset,n)
    evalcb = throttle(() -> @show(accuracy(X, Y)), 10)
    opt = ADAM(params(vgg_24))
    for i=1:n
        Flux.train!(loss, dataset, opt, cb = evalcb)
    end
end

function test_VGG24(X, Y)
    accuracy(X, Y)
end

test_VGG24 (generic function with 1 method)

In [45]:
#train_x_all = convert(Array{Float64,4}, train_x_all);
batches=[(train_x_all[:,:,:,i],train_y[:,i]) for i in partition(1:size(train_x_all,4),10)] ;

In [46]:
train_VGG24(train_x_all,train_y,batches,1)

accuracy(X, Y) = 0.50625
accuracy(X, Y) = 0.825
accuracy(X, Y) = 0.81875
accuracy(X, Y) = 0.9125
accuracy(X, Y) = 0.84375
accuracy(X, Y) = 0.975


In [47]:
test_VGG24(test_x_all,test_y)

0.875

In [26]:
params(vgg_24)

4-element Array{Any,1}:
 Flux.Tracker.TrackedReal{Float32}[NaN (tracked) NaN (tracked) … NaN (tracked) NaN (tracked); NaN (tracked) NaN (tracked) … NaN (tracked) NaN (tracked); … ; NaN (tracked) NaN (tracked) … NaN (tracked) NaN (tracked); NaN (tracked) NaN (tracked) … NaN (tracked) NaN (tracked)]                                                               
 Flux.Tracker.TrackedReal{Float32}[NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked)  …  NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked), NaN (tracked)]
 Flux.Tracker.TrackedReal{Float64}[NaN (tracked) NaN (tracked) … NaN (tracked) NaN (tracked); NaN (tracked) NaN (tracked) … NaN (tracked) NaN (tracked)]                                                                                                                                                    

#### 4.Now, make a change in vgg_24 at Q3.2 by adding the following: initlitize the w and b of the last Dense (before
the softmax) with the optimal weights that you reached at question 2 from vgg_26
5.Train and Test again and show if there is an improvement in the training time and in the final testing accuracy

In [36]:
dense_layer_w= params(vgg_26)[1]
dense_layer_b= params(vgg_26)[2]

function my_last_dense_w(out,in_n)
    Tracker.data(dense_layer_w)
end

function my_last_dense_b(b)
    Tracker.data(dense_layer_b)
end


my_last_dense_b (generic function with 2 methods)

In [39]:
vgg_24_2 = Chain(
    VGG19().layers[1:24],
    Dense(4096, 4096,relu, initW= my_init_w, initb= my_init_b),
    Dense(4096, 2, initW= my_last_dense_w, initb= my_last_dense_b),
    softmax
)

Chain(Chain(Conv((3, 3), 3=>64, NNlib.relu), Conv((3, 3), 64=>64, NNlib.relu), getfield(Metalhead, Symbol("##42#48"))(), Conv((3, 3), 64=>128, NNlib.relu), Conv((3, 3), 128=>128, NNlib.relu), getfield(Metalhead, Symbol("##43#49"))(), Conv((3, 3), 128=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), Conv((3, 3), 256=>256, NNlib.relu), getfield(Metalhead, Symbol("##44#50"))(), Conv((3, 3), 256=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), getfield(Metalhead, Symbol("##45#51"))(), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), Conv((3, 3), 512=>512, NNlib.relu), getfield(Metalhead, Symbol("##46#52"))(), getfield(Metalhead, Symbol("##47#53"))(), Dense(25088, 4096, NNlib.relu), Dropout{Float32}(0.5f0, false)), Dense(4096, 4096, NNlib.relu), Dense(4096, 2), NNlib.softmax)

In [40]:
loss(x, y) = crossentropy(vgg_24_2(x), y)
accuracy(x, y) = mean(onecold(vgg_24_2(x)) .== onecold(y))

function train_VGG24_2(X, Y, dataset,n)
    evalcb = throttle(() -> @show(accuracy(X, Y)), 10)
    opt = ADAM(params(vgg_24_2))
    for i=1:n
        Flux.train!(loss, dataset, opt, cb = evalcb)
    end
end

function test_VGG24_2(X, Y)
    accuracy(X, Y)
end

test_VGG24_2 (generic function with 1 method)

In [41]:
train_VGG24_2(train_x_all,train_y,batches,1)

accuracy(X, Y) = 0.5
accuracy(X, Y) = 0.80625
accuracy(X, Y) = 0.9
accuracy(X, Y) = 0.95
accuracy(X, Y) = 0.9125
accuracy(X, Y) = 0.9625


In [42]:
test_VGG24(test_x_all,test_y)

0.9